# **FEATURE ENGINEER**

### Esta Jupyter Notebook está dedicada a la preparación de los conjuntos de datos necesarios para llevar a cabo consultas en la API.

### IMPORTACIÓN DE LIBRERÍAS 

In [1]:
import pandas as pd
import numpy as np 
import json
import ast 
import re
import pyarrow as pa
import pyarrow.parquet as pq
import fastparquet as fp 
from textblob import TextBlob
import nltk
from fastapi import FastAPI
import os

%load_ext autoreload
%autoreload 2

In [2]:
df_reviews = pd.read_parquet("data/user_reviews_clean.parquet")
df_games= pd.read_parquet('data/user_games_clean.parquet')
df_items = pd.read_parquet('data/user_items_clean.parquet')

In [3]:
df_games

,publisher,genres,app_name,title,url,release_date,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,Año_estreno
0,Kotoshiro,Action,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
0,Kotoshiro,Casual,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
0,Kotoshiro,Indie,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
0,Kotoshiro,Simulation,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
0,Kotoshiro,Strategy,Lost Summoner Kitty,Lost Summoner Kitty,http://store.steampowered.com/app/761140/Lost_...,2018-01-04,"[Strategy, Action, Indie, Casual, Simulation]",http://steamcommunity.com/app/761140/reviews/?...,4.49,[Single-player],4.99,False,761140,NaN,Kotoshiro,2018
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32132,Laush Studio,Indie,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,2018
32132,Laush Studio,Racing,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,2018
32132,Laush Studio,Simulation,Russian Roads,Russian Roads,http://store.steampowered.com/app/610660/Russi...,2018-01-04,"[Indie, Simulation, Racing]",http://steamcommunity.com/app/610660/reviews/?...,1.39,"[Single-player, Steam Achievements, Steam Trad...",1.99,False,610660,NaN,Laush Dmitriy Sergeevich,2018
32133,SIXNAILS,Casual,EXIT 2 - Directions,EXIT 2 - Directions,http://store.steampowered.com/app/658870/EXIT_...,2017-09-02,"[Indie, Casual, Puzzle, Singleplayer, Atmosphe...",http://steamcommunity.com/app/658870/reviews/?...,NaN,"[Single-player, Steam Achievements, Steam Cloud]",4.99,False,658870,NaN,"xropi,stev3ns",2017


In [4]:
df_items

,item_id,item_name,playtime_forever,steam_id,items_count,user_id,user_url
0,10,Counter-Strike,6,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
1,20,Team Fortress Classic,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
2,30,Day of Defeat,7,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
3,40,Deathmatch Classic,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
4,50,Half-Life: Opposing Force,0,76561197970982479,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...
...,...,...,...,...,...,...,...
5153204,346330,BrainBread 2,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153205,373330,All Is Dust,0,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153206,388490,One Way To Die: Steam Edition,3,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...
5153207,521570,You Have 10 Seconds 2,4,76561198329548331,7,76561198329548331,http://steamcommunity.com/profiles/76561198329...


In [5]:
df_reviews

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.
2,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-04-21,,43110,No ratings yet,True,Great atmosphere. The gunplay can be a bit chu...
3,js41637,http://steamcommunity.com/id/js41637,,2014-06-24,,251610,15 of 20 people (75%) found this review helpful,True,I know what you think when you see this title ...
4,js41637,http://steamcommunity.com/id/js41637,,2013-09-08,,227300,0 of 1 people (0%) found this review helpful,True,For a simple (it's actually not all that simpl...
...,...,...,...,...,...,...,...,...,...
49181,llDracuwulf,http://steamcommunity.com/id/llDracuwulf,1 person found this review funny,2015-10-14,,730,1 of 1 people (100%) found this review helpful,True,its FUNNNNNNNN
49182,76561198223837952,http://steamcommunity.com/profiles/76561198223...,,2015-10-10,,253980,No ratings yet,True,Awesome fantasy game if you don't mind the gra...
49183,76561198229845636,http://steamcommunity.com/profiles/76561198229...,,2015-10-31,,730,No ratings yet,True,Prettyy Mad Game
49184,76561198232478272,http://steamcommunity.com/profiles/76561198232...,,2015-12-14,,730,No ratings yet,True,AMAZING GAME 10/10


In [25]:
file_path = os.path.abspath('data/user_reviews_clean.parquet')
df_reviews = pd.read_parquet(file_path)
file_path 

'c:\\Users\\ACER\\Documents\\HENRY\\LABS\\PI01\\JUPYTER NOTEBOOKS\\data\\user_reviews_clean.parquet'

### **Sentiment_analysis( año : int ):** 
Claro, aquí tienes una explicación concisa de lo que hace el código:

En el DataFrame `user_reviews_clean`, se crea una nueva columna llamada "sentiment_analysis". Luego, se aplica un análisis de sentimientos utilizando la biblioteca NLP (Natural Language Toolkit) o TextBlob. El objetivo de este análisis es determinar la polaridad del texto de las reseñas, es decir, si es positivo, negativo o neutro.

La escala utilizada es la siguiente:
- 0: Negativo
- 1: Neutro
- 2: Positivo

Este análisis se realiza específicamente en la columna "review" del DataFrame. Una vez que se ha realizado el análisis de sentimientos y se han asignado los valores correspondientes en la columna "sentiment_analysis", la columna original "review" se elimina del DataFrame. En resumen, este código agrega una columna de análisis de sentimiento a las reseñas, utiliza una escala de 0 a 2 para representar el sentimiento y luego elimina la columna de reseñas originales.

In [26]:
from textblob import TextBlob

def sentiment_analysis(texto):
    # Se calcula el sentimiento del texto
    analysis = TextBlob(texto)
    # El atributo sentiment.polarity devuelve la polaridad del sentimiento (entre -1 y 1)
    polaridad = analysis.sentiment.polarity
    return polaridad

def puntaje(polaridad):
    # Asigna 0 si la polaridad es negativa, 1 si es neutral y 2 si es positiva
    if polaridad < -0.1:
        return 0  # Negativo
    elif polaridad > 0.1:
        return 2  # Positivo
    else:
        return 1  # Neutral



In [27]:
df_sentiment = df_reviews.copy()
# Aplicar la función de análisis de sentimiento a la columna 'review' y crear una nueva columna 'polaridad'
df_sentiment['polaridad'] = df_sentiment['review'].astype(str).apply(sentiment_analysis)

# Aplicar la función de puntaje a la columna 'polaridad' y crear una nueva columna 'sentimiento'
df_sentiment['sentimiento'] = df_sentiment['polaridad'].apply(puntaje)


In [28]:
df_sentiment.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review,polaridad,sentimiento
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...,0.174444,2
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.,0.337500,2


# **FUNCIONES PARA EL DESARROLLO DE LA API**

### 1. Def userdata( User_id : str ): 
Debe devolver cantidad de dinero gastado por el usuario, el porcentaje de recomendación en base a reviews.recommend y cantidad de items.

In [6]:
df_games['id'] = pd.to_numeric(df_games['id'], errors='coerce')
df_items['item_id'] = pd.to_numeric(df_items['item_id'], errors='coerce')

In [7]:
columns_drop = ['steam_id', 'item_name']
df_items.drop(columns=columns_drop, inplace=True)

In [8]:
df_itgame = df_items.merge(df_games, left_on='item_id', right_on='id', how='left')

In [9]:
df_mix = df_itgame[['user_id','price']]
df_mix.dropna(inplace=True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_4640\3293973895.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_mix.dropna(inplace=True)


In [10]:
df_recomendacion = df_reviews[['user_id', 'recommend']]
df_conteo = df_items[['user_id', 'items_count']]
df_conteo.drop_duplicates(inplace=True)

C:\Users\ACER\AppData\Local\Temp\ipykernel_4640\2336736842.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_conteo.drop_duplicates(inplace=True)


In [11]:
def userdata(user_id):
    """
    Esta función calcula y devuelve información sobre un usuario en función de su ID.

    Argumentos:
    user_id (str): ID de identificación del usuario.

    Retorna:
    dict: Un diccionario con la información solicitada.
    """
    user_data = df_mix[df_mix['user_id'] == user_id]
    
    if user_data.empty:
        return {'Usuario': user_id, 'Mensaje': 'Usuario no encontrado'}
    
    gasto = round(user_data['price'].sum(), 2)
    ttl_reviews = user_data.shape[0]
    ttl_recommendations = user_data['recommend'].sum()
    ttl_items = df_conteo[df_conteo['user_id'] == user_id]['items_count'].iloc[0]

    return {
        'Usuario': user_id,
        'Cantidad de dinero gastado': gasto,
        'Porcentaje de recomendación': round((ttl_recommendations / ttl_reviews) * 100, 2),
        'Cantidad de items': ttl_items
    }


In [ ]:
userdata

### 2. Def countreviews( YYYY-MM-DD y YYYY-MM-DD : str ): 
Cantidad de usuarios que realizaron reviews entre las fechas dadas y, el porcentaje de recomendación de los mismos en base a reviews.recommend.

In [35]:
df_countreviews = df_reviews.merge(df_items, on='user_id', how='inner')

df_countreviews = df_countreviews.loc[:, ['user_id', 'posted', 'recommend']]
df_countreviews

,user_id,posted,recommend
0,76561197970982479,2011-11-05,True
1,76561197970982479,2011-11-05,True
2,76561197970982479,2011-11-05,True
3,76561197970982479,2011-11-05,True
4,76561197970982479,2011-11-05,True
...,...,...,...
6225703,76561198232478272,2015-12-14,True
6225704,76561198232478272,2015-12-14,True
6225705,76561198232478272,2015-12-14,True
6225706,76561198232478272,2015-12-14,True


In [36]:
def count_reviews(Start, End):
    '''
    Calcula la cantidad de usuarios que realizaron reviews entre las fechas dadas y 
    el porcentaje de recomendacion de esos usuarios.
    
    Argumentos:
    inicio (str): Fecha de inicio del periodo a evaluar.
    fin (str): Fecha de fin del periodo a evaluar
    '''
    Start = pd.to_datetime(Start)
    End = pd.to_datetime(End) 
    
    reviews_entre_fechas = df_countreviews[df_countreviews['posted'].between(Start, End)]
    
    cantidad_usuarios = len(reviews_entre_fechas['user_id'].unique())
    
    porcentaje_recomendacion = (reviews_entre_fechas['recommend'].sum() / len(reviews_entre_fechas)) * 100
    
    return {'Cantidad de usuarios:': cantidad_usuarios,
            'Porcentaje de recomendación:': round(porcentaje_recomendacion, 2)
           }


In [37]:
count_reviews('2011-05-08','2011-10-10')

{'Cantidad de usuarios:': 184, 'Porcentaje de recomendación:': 97.57}

**PLAYTIME_FOREVER POR USUARIO**

Se desea crear un DataFrame que agrupe el tiempo de juego por género de videojuego, asociando cada usuario con su ID y URL de perfil. Para lograrlo, se seguirán varios pasos, incluyendo la creación de DataFrames auxiliares.

Primero, se extraerán las columnas 'playtime_forever', 'user_id' y 'item_id' del DataFrame df_items y se guardarán en otro DataFrame para su posterior uso.

In [ ]:
# Supongamos que tienes una columna 'user_id' en df_playtime_genre y 'item_id' en df_posicion
# También supongamos que quieres combinar en función de esas columnas

# Utiliza merge para combinar las tablas en base a las columnas 'user_id' y 'item_id'
df_reco = df_playtime_genre.merge(df_posicion[['user_id', 'item_id', 'ranking']], on=['user_id', 'item_id'], how='inner')

# df_reco ahora contiene las columnas 'user_id', 'item_id' y 'ranking'


In [38]:
df_playtime_genre = df_items[['playtime_forever', 'user_id', 'item_id']]
df_playtime_genre

,playtime_forever,user_id,item_id
0,6,76561197970982479,10
1,0,76561197970982479,20
2,7,76561197970982479,30
3,0,76561197970982479,40
4,0,76561197970982479,50
...,...,...,...
5153204,0,76561198329548331,346330
5153205,0,76561198329548331,373330
5153206,3,76561198329548331,388490
5153207,4,76561198329548331,521570


Del DataFrame df_games, se obtiene el 'id' del artículo (item) y el género del videojuego.

In [39]:
genre_item = df_games[['genres', 'id']]
# Se renombra la columna 'id' para unirla con el dataframe anterior
genre_item = genre_item.rename(columns={'id':'item_id'})
genre_item

,genres,item_id
0,Action,761140
0,Casual,761140
0,Indie,761140
0,Simulation,761140
0,Strategy,761140
...,...,...
32132,Indie,610660
32132,Racing,610660
32132,Simulation,610660
32133,Casual,658870


A continuación, se procede a combinar las dos tablas previamente mencionadas para obtener los géneros de todos los usuarios, junto con sus identificadores (ID) y enlaces (URL).

In [40]:
df_playtime_genre = df_playtime_genre.merge(genre_item.rename(columns={'genre': 'genre'}), on='item_id')
df_playtime_genre

,playtime_forever,user_id,item_id,genres
0,6,76561197970982479,10,Action
1,0,js41637,10,Action
2,0,Riot-Punch,10,Action
3,93,doctr,10,Action
4,108,corrupted_soul,10,Action
...,...,...,...,...
9877299,164,76561198107283457,354280,Indie
9877300,164,76561198107283457,354280,Simulation
9877301,0,inven,433920,Adventure
9877302,0,inven,433920,Indie


Se agrupa por género y usuario y se suma el tiempo de juego de para cada caso, se asume que la columna playtime_forever se encuentra en minutos, por lo que se transforma a horas.

In [41]:
# Se agrupa por usuario y se suma el tiempo de juego
aggru_genero = df_playtime_genre.groupby(['genres', 'user_id'])['playtime_forever'].sum().reset_index()
# Se transforma la columna 'playtime_forever' a horas
aggru_genero['playtime_horas'] = aggru_genero['playtime_forever']/60
# Se borra la columna en minutos
aggru_genero = aggru_genero.drop('playtime_forever', axis=1)
aggru_genero

,genres,user_id,playtime_horas
0,Action,--000--,2324.483333
1,Action,--ace--,1155.416667
2,Action,--ionex--,638.583333
3,Action,-2SV-vuLB-Kg,708.333333
4,Action,-404PageNotFound-,1957.050000
...,...,...,...
671349,Web Publishing,zepavil,632.100000
671350,Web Publishing,zeshirky,0.016667
671351,Web Publishing,zevlupine,0.066667
671352,Web Publishing,zilaman,0.150000


Se busca obtener la URL del perfil de cada usuario, para lograrlo, se crea un DataFrame auxiliar que contiene las columnas 'user_id' y 'user_url'. Luego, se eliminan las filas duplicadas para garantizar que cada usuario aparezca solo una vez. Por último, este DataFrame auxiliar se agrega al DataFrame anterior para crear el DataFrame final llamado 'df_playtime_forever', que incluye las horas de juego por género, el ID de usuario y su URL de perfil.

In [42]:
# Crear un DataFrame 'user_url_usuario' con las columnas 'user_url' y 'user_id' desde df_items
user_url_usuario = df_items[['user_url', 'user_id']]
# Eliminar duplicados en 'user_url_usuario' para obtener una sola entrada por 'user_id'
user_url_usuario = user_url_usuario.drop_duplicates(subset='user_id', keep='first')
# Realizar una fusión (merge) izquierda entre 'agg_genero' y 'user_url_usuario' utilizando 'user_id' como clave de unión
df_playtime_forever = aggru_genero.merge(user_url_usuario, on='user_id', how='left')
# Mostrar las primeras filas del DataFrame resultante
df_playtime_forever.head()


,genres,user_id,playtime_horas,user_url
0,Action,--000--,2324.483333,http://steamcommunity.com/id/--000--
1,Action,--ace--,1155.416667,http://steamcommunity.com/id/--ace--
2,Action,--ionex--,638.583333,http://steamcommunity.com/id/--ionex--
3,Action,-2SV-vuLB-Kg,708.333333,http://steamcommunity.com/id/-2SV-vuLB-Kg
4,Action,-404PageNotFound-,1957.050000,http://steamcommunity.com/id/-404PageNotFound-


### **POSICIÓN DE GÉNEROS POR TIEMPO**

En este momento, el objetivo es crear un DataFrame adicional que muestre la clasificación de géneros de videojuegos según el tiempo de juego acumulado en cada uno. Aprovechamos el hecho de que ya tenemos un DataFrame auxiliar que contiene información sobre géneros y el tiempo jugado, y lo utilizamos para este propósito. El proceso implica los siguientes pasos:

- Primero, agrupamos los datos por género.
- Luego, ordenamos los géneros en función del tiempo de juego.
- Por último, agregamos una columna que muestre el ranking de géneros.

In [234]:
df_posicion = aggru_genero.groupby('genres')['playtime_horas'].sum().reset_index()
# Se ordena por 'playtime_horas'
df_posicion = df_posicion.sort_values(by='playtime_horas', ascending=False)
# Agregar una columna con la posición del ranking
df_posicion['ranking'] = df_posicion['playtime_horas'].rank(ascending=False).astype(int)
df_posicion

,genres,playtime_horas,ranking
0,Action,5.125167e+07,1
9,Indie,2.459123e+07,2
12,RPG,1.713213e+07,3
1,Adventure,1.497939e+07,4
14,Simulation,1.425438e+07,5
17,Strategy,1.084995e+07,6
8,Free to Play,1.006072e+07,7
10,Massively Multiplayer,7.352093e+06,8
4,Casual,4.155277e+06,9
6,Early Access,2.611542e+06,10


### 3. Def genre( género : str ): 
Devuelve el puesto en el que se encuentra un género sobre el ranking de los mismos analizado bajo la columna PlayTimeForever.

In [230]:
df_posicion

,genres,playtime_horas,ranking
0,Action,5.125167e+07,1
9,Indie,2.459123e+07,2
12,RPG,1.713213e+07,3
1,Adventure,1.497939e+07,4
14,Simulation,1.425438e+07,5
17,Strategy,1.084995e+07,6
8,Free to Play,1.006072e+07,7
10,Massively Multiplayer,7.352093e+06,8
4,Casual,4.155277e+06,9
6,Early Access,2.611542e+06,10


In [44]:
def genre(genero):
    # Busca el ranking para el género de interés
    rank = df_posicion[df_posicion['genres'] == genero]['ranking'].iloc[0]
    return {
        'rank': rank
    }

In [231]:
genero = 'Early Access'
genre(genero)

{'rank': 10}

 ### 4. Def userforgenre( género : str ):

In [46]:
def userforgenre(genero):
    # Filtra el DataFrame principal (df_playtime_forever) para obtener datos relacionados con el género especificado.
    data_por_genero = df_playtime_forever[df_playtime_forever['genres'] == genero]
    
    # Agrupa los datos filtrados por usuario y suma las horas de juego para cada uno.
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()
    
    # Crea un diccionario para almacenar la información de los mejores usuarios.
    top_users_dict = {}
    
    # Itera a través de los datos de los mejores usuarios y los agrega al diccionario.
    for index, row in top_users.iterrows():
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users_dict[index + 1] = user_info
    
    # Devuelve el diccionario de los mejores usuarios.
    return top_users_dict


In [49]:
genero = 'Action'
userforgenre(genero)

{1: {'user_id': 'Sp3ctre', 'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
 2: {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 3: {'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 4: {'user_id': 'Terminally-Chill',
  'user_url': 'http://steamcommunity.com/id/Terminally-Chill'},
 5: {'user_id': 'DownSyndromeKid',
  'user_url': 'http://steamcommunity.com/id/DownSyndromeKid'}}

In [25]:
def userforgenre(genre):
    '''
    Devuelve el TOP 5 de usuarios con más horas jugadas en un género específico.

    Argumento:
    genre (str): El género de juegos para el que se desea obtener el TOP 5 de usuarios.
    
    '''

    # Filtrar el DataFrame para obtener datos específicos del género y tomar los primeros 5 registros
    top5 = df_ugenre[df_ugenre['genres'] == genre].nlargest(5, 'playtime_forever')

    # Crear una lista de resultados en el formato deseado
    result = [
        'El TOP 5 de usuarios para el género,',
        genre,
        'es el siguiente'
    ]

    # Agregar cada línea del DataFrame al resultado como una cadena
    for index, row in top5.iterrows():
        result.append(f"{row['genres']}\t{row['user_id']}\t{row['user_url']}\t{row['playtime_forever']}")

    return result



In [26]:
userforgenre('Action')

['El TOP 5 de usuarios para el género,',
 'Action',
 'es el siguiente',
 'Action\tSp3ctre\thttp://steamcommunity.com/id/Sp3ctre\t1699307',
 'Action\tshinomegami\thttp://steamcommunity.com/id/shinomegami\t1580428',
 'Action\tTerminally-Chill\thttp://steamcommunity.com/id/Terminally-Chill\t1065742',
 'Action\tDownSyndromeKid\thttp://steamcommunity.com/id/DownSyndromeKid\t1061193',
 'Action\tstopgovtcorruption\thttp://steamcommunity.com/id/stopgovtcorruption\t1047411']

### 5. Def developer( desarrollador : str ):  
Cantidad de items y porcentaje de contenido Free por año según empresa desarrolladora. Ejemplo de salida:

- Activision	
- Año	Contenido Free
- 2023	 27%
- 2022	 25%
- xxxx	 xx%

In [82]:
df_itgame.head(5)

,item_id,playtime_forever,items_count,user_id,user_url,publisher,genres,app_name,title,url,...,tags,reviews_url,discount_price,specs,price,early_access,id,metascore,developer,Año_estreno
0,10,6,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Valve,Action,Counter-Strike,Counter-Strike,http://store.steampowered.com/app/10/CounterSt...,...,"[Action, FPS, Multiplayer, Shooter, Classic, T...",http://steamcommunity.com/app/10/reviews/?brow...,NaN,"[Multi-player, Valve Anti-Cheat enabled]",9.99,False,10.0,88.0,Valve,2000
1,20,0,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Valve,Action,Team Fortress Classic,Team Fortress Classic,http://store.steampowered.com/app/20/Team_Fort...,...,"[Action, FPS, Multiplayer, Classic, Shooter, C...",http://steamcommunity.com/app/20/reviews/?brow...,NaN,"[Multi-player, Valve Anti-Cheat enabled]",4.99,False,20.0,NaN,Valve,1999
2,30,7,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Valve,Action,Day of Defeat,Day of Defeat,http://store.steampowered.com/app/30/Day_of_De...,...,"[FPS, World War II, Multiplayer, Action, Shoot...",http://steamcommunity.com/app/30/reviews/?brow...,NaN,"[Multi-player, Valve Anti-Cheat enabled]",4.99,False,30.0,79.0,Valve,2003
3,40,0,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Valve,Action,Deathmatch Classic,Deathmatch Classic,http://store.steampowered.com/app/40/Deathmatc...,...,"[Action, FPS, Multiplayer, Classic, Shooter, F...",http://steamcommunity.com/app/40/reviews/?brow...,NaN,"[Multi-player, Valve Anti-Cheat enabled]",4.99,False,40.0,NaN,Valve,2001
4,50,0,277,76561197970982479,http://steamcommunity.com/profiles/76561197970...,Valve,Action,Half-Life: Opposing Force,Half-Life: Opposing Force,http://store.steampowered.com/app/50/HalfLife_...,...,"[FPS, Action, Sci-fi, Singleplayer, Classic, S...",http://steamcommunity.com/app/50/reviews/?brow...,NaN,"[Single-player, Multi-player, Valve Anti-Cheat...",4.99,False,50.0,NaN,Gearbox Software,1999


In [83]:
df_itgame_prueba = df_itgame[['item_id', 'developer', 'price', 'release_date']]
df_itgame_prueba = df_itgame_prueba.dropna().drop_duplicates()
df_itgame_prueba.head()


,item_id,developer,price,release_date
0,10,Valve,9.99,2000-11-01
1,20,Valve,4.99,1999-04-01
2,30,Valve,4.99,2003-05-01
3,40,Valve,4.99,2001-06-01
4,50,Gearbox Software,4.99,1999-11-01


In [85]:
df_ITGA = df_itgame_prueba.copy()
df_ITGA['release_date'] = pd.to_datetime(df_ITGA['release_date'], errors='coerce')

df_ITGA

,item_id,developer,price,release_date
0,10,Valve,9.99,2000-11-01
1,20,Valve,4.99,1999-04-01
2,30,Valve,4.99,2003-05-01
3,40,Valve,4.99,2001-06-01
4,50,Gearbox Software,4.99,1999-11-01
...,...,...,...,...
9516552,456780,Milestone S.r.l.,9.99,2016-06-09
9944500,444770,Silicon Jelly,7.99,2016-03-22
10248553,375450,"KOEI TECMO GAMES CO., LTD.",59.99,2016-03-23
10696631,354280,CashcowGames,5.99,2016-02-23


In [90]:
def developer(desarrollador):
    '''
    Devuelve la cantidad de elementos y el porcentaje de contenido gratuito por año según la empresa desarrolladora.

    Argumento:
    desarrollador (str): El desarrollador del juego (elemento) para el cual se desean obtener los valores mencionados. 
    '''
    
    # Filtrar el DataFrame por el desarrollador especificado
    developer_df = df_ITGA[df_ITGA['developer'] == desarrollador]
    
    # Crear un DataFrame para elementos gratuitos
    elementos_gratuitos = developer_df[developer_df['price'] == 0]
    
    # Agrupar y contar elementos totales por año
    elementos_totales_por_anio = developer_df.groupby(developer_df['release_date'].dt.year)['item_id'].nunique()
    
    # Agrupar y contar elementos gratuitos por año
    elementos_gratuitos_por_anio = elementos_gratuitos.groupby(elementos_gratuitos['release_date'].dt.year)['item_id'].nunique()
    
    # Calcular el porcentaje de contenido gratuito por año
    porcentaje_gratuito_por_anio = (elementos_gratuitos_por_anio / elementos_totales_por_anio) * 100
    
    # Crear un DataFrame con los resultados
    resultados = pd.DataFrame({
        'Año': elementos_totales_por_anio.index,
        'Cantidad de Elementos': elementos_totales_por_anio.values,
        'Porcentaje Gratuito': porcentaje_gratuito_por_anio.values
    })
    
    # Devolver los resultados como una lista de registros (diccionarios)
    return resultados.to_dict(orient='records')


In [91]:
desarrollador = 'Valve'
developer(desarrollador)

[{'Año': 1998, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan},
 {'Año': 1999, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan},
 {'Año': 2000, 'Cantidad de Elementos': 2, 'Porcentaje Gratuito': nan},
 {'Año': 2001, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan},
 {'Año': 2003, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan},
 {'Año': 2004, 'Cantidad de Elementos': 5, 'Porcentaje Gratuito': nan},
 {'Año': 2006, 'Cantidad de Elementos': 2, 'Porcentaje Gratuito': nan},
 {'Año': 2007, 'Cantidad de Elementos': 2, 'Porcentaje Gratuito': nan},
 {'Año': 2008, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan},
 {'Año': 2009, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan},
 {'Año': 2010, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan},
 {'Año': 2011, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan},
 {'Año': 2012, 'Cantidad de Elementos': 1, 'Porcentaje Gratuito': nan}]

### 6. Def sentiment_analysis( año : int )

In [189]:
df_funsentiment = df_sentiment.merge(df_items, on='user_id', how='inner') 


In [190]:
df_funsentiment.drop(columns=['item_id_y','playtime_forever'],inplace=True)

In [94]:
df_funsentiment.drop_duplicates(inplace=True)

In [191]:
df_funsentiment = df_funsentiment[['sentimiento','posted']]

In [192]:
# Eliminamos la palabra 'Posted:'
df_funsentiment['posted']= df_funsentiment['posted'].replace({'Posted':''},regex=True)

# Convertimos la columna a datetime.
df_funsentiment['posted'] = pd.to_datetime(df_funsentiment['posted'],errors='coerce')

# Eliminamos los valores con error (NaT), por ejemplo, fechas que no tenían el año.
df_funsentiment.dropna(inplace=True)


In [197]:
df_funsentiment['anio'] = df_funsentiment['posted'].dt.year
df_funsentiment

,sentimiento,posted,anio
0,2,2011-11-05,2011
1,2,2011-11-05,2011
2,2,2011-11-05,2011
3,2,2011-11-05,2011
4,2,2011-11-05,2011
...,...,...,...
6225703,2,2015-12-14,2015
6225704,2,2015-12-14,2015
6225705,2,2015-12-14,2015
6225706,2,2015-12-14,2015


In [133]:
df_funsentiment.head(2)

,sentimiento,posted,anio
0,2,2011-11-05,1970
277,2,2011-07-15,1970


In [136]:
df_reviews.head(2)

,user_id,user_url,funny,posted,last_edited,item_id,helpful,recommend,review
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-11-05,,1250,No ratings yet,True,Simple yet with great replayability. In my opi...
1,76561197970982479,http://steamcommunity.com/profiles/76561197970...,,2011-07-15,,22200,No ratings yet,True,It's unique and worth a playthrough.


In [227]:
def sentiment_analysis(anio):
    ''' 
    Devuelve un DataFrame con la cantidad de registros de reseñas de usuarios categorizados por análisis de sentimiento para un año específico.
    
    Argumentos:
    anio (int): Año en el cual queremos obtener el análisis de sentimiento.
    df_f6 (DataFrame): El DataFrame que contiene los datos de reseñas y sentimientos.
   
    '''

    # Filtramos el DataFrame según el año definido como argumento
    df_filtered = df_funsentiment[df_funsentiment['posted'].dt.year == anio]
    
    # Utilizamos value_counts() para contar los valores únicos en la columna 'sentimiento'
    positivos = 0
    neutros = 0
    negativos = 0

    # Obtenemos la cantidad de valores positivos, negativos y neutros
    for i in df_filtered['sentimiento']:
        if i == 2:
            positivos += 1
        elif i == 1:
            neutros += 1
        elif i == 0:
            negativos += 1 

    # Crear un DataFrame con los resultados
    resultado = pd.DataFrame({
        'Positivos': [positivos],
        'Neutros': [neutros],
        'Negativos': [negativos]
    })

    return resultado


In [229]:
sentiment_analysis(2013)

,Positivos,Neutros,Negativos
0,437955,304350,103205


### **CARGA DEL CONJUNTO DE DATOS**

In [41]:
Save = 'data/df_recomendacion.csv'
df_recomendacion.to_csv(Save, index=False, encoding='utf-8')

In [42]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_recomendacion.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_recomendacion)
pq.write_table(table,output_file)

In [43]:
Save = 'data/df_mix.csv'
df_mix.to_csv(Save, index=False, encoding='utf-8')

In [44]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_mix.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_mix)
pq.write_table(table,output_file)

In [45]:
Save = 'data/df_conteo.csv'
df_conteo.to_csv(Save, index=False, encoding='utf-8')

In [46]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_conteo.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_conteo)
pq.write_table(table,output_file)

In [47]:
Save = 'data/df_posicion.csv'
df_posicion.to_csv(Save, index=False, encoding='utf-8')

In [48]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_posicion.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_conteo)
pq.write_table(table,output_file)

In [49]:
Save = 'data/df_ugenre.csv'
df_ugenre.to_csv(Save, index=False, encoding='utf-8')

In [50]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_ugenre.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_ugenre)
pq.write_table(table,output_file)

In [51]:
Save = 'data/df_funsentiment.csv'
df_funsentiment.to_csv(Save, index=False, encoding='utf-8')

In [52]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_funsentiment.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_funsentiment)
pq.write_table(table,output_file)

In [53]:
Save = 'data/df_sentiment.csv'
df_sentiment.to_csv(Save, index=False, encoding='utf-8')

In [ ]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_sentiment.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_sentiment)
pq.write_table(table,output_file)

In [ ]:
Save = 'data/df_ITGA.csv'
df_ITGA.to_csv(Save, index=False, encoding='utf-8')

In [ ]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_ITGA.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_ITGA)
pq.write_table(table,output_file)

In [ ]:
Save = 'data/df_countreviews.csv'
df_countreviews.to_csv(Save, index=False, encoding='utf-8')

In [ ]:
#Indico donde quiero guardar el parquet y con que nombre
output_file= "data/df_countreviews.parquet"

#Transformo a traves de una tabla el archivo csv en parquet
table = pa.Table.from_pandas(df_ITGA)
pq.write_table(table,output_file)